In [1]:
import pandas as pd
import glob
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bert_score import score
import markdown
from bs4 import BeautifulSoup

In [2]:
# Load the CSV file containing the generated answers from the fine-tuned model
finetuned_answers = pd.read_csv('/home/subin/Desktop/subin/finetuned_model_answers_2.csv')  # Adjust path if needed

# Function to extract text from Markdown files
def extract_text_from_markdown(file_path):
    with open(file_path, 'r') as file:
        md_content = file.read()
    # Convert Markdown to HTML
    html_content = markdown.markdown(md_content)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    # Extract and return plain text
    return soup.get_text()

# Load and extract text from all Markdown files in the reference folder
markdown_folder = '/home/subin/Desktop/subin/ritsu_bot/markdown_files'
all_reference_texts = {}
for file_path in glob.glob(os.path.join(markdown_folder, '*.md')):
    filename = os.path.basename(file_path)
    all_reference_texts[filename] = extract_text_from_markdown(file_path).strip()

# Extract reference texts and generated answers for BERTScore
references = []
hypotheses = finetuned_answers['Answer'].str.strip().tolist()

for index, row in finetuned_answers.iterrows():
    filename = row['Filename']
    if filename in all_reference_texts:
        reference_text = all_reference_texts[filename]
    else:
        print(f"Warning: {filename} not found in markdown folder")
        reference_text = ""  # Assign empty text or handle as needed
    references.append(reference_text)

# Filter out empty references and corresponding hypotheses
filtered_references = [ref for ref in references if ref]
filtered_hypotheses = [hyp for ref, hyp in zip(references, hypotheses) if ref]

In [3]:
# Calculate BERTScore
P, R, F1 = score(filtered_hypotheses, filtered_references, lang="en", verbose=True)

# Average F1 score
average_f1 = F1.mean().item()
print(f"Average BERTScore F1: {average_f1}")

# Add the BERT F1 scores to the DataFrame
finetuned_answers['BERTScore F1'] = F1.tolist() + [None] * (len(hypotheses) - len(F1))

# Save the results to a new CSV file
output_csv_path = '/home/subin/Desktop/subin/BERTScore_finetuned.csv'
finetuned_answers.to_csv(output_csv_path, index=False)

print(f"BERTScore results saved to {output_csv_path}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 9.61 seconds, 20.30 sentences/sec
Average BERTScore F1: 0.827528178691864
BERTScore results saved to /home/subin/Desktop/subin/BERTScore_finetuned.csv
